### 名前マンダリング

In [ ]:
class MyClass:
    __secret_value = 1

# privateキーワードの名前にアクセスするとAttributeError例外が送出される
instance_of = MyClass()
try:
    print(instance_of.__secret_value)
except Exception as ex:
    print(ex)

print(dir(instance_of))

### ディスクリプタ

In [ ]:
class RevealAccess:
    """値を通常通り設定・返すデータディスクリプタ
    アクセスログも出力する
    """
    def __init__(self, initval=None, name='実数'):
        self.val = initval
        self.name = name
    
    def __get__(self, obj, objtype):
        print('取得', self.name)
        return self.val
    
    def __set__(self, obj, val):
        print('更新', self.name)
        self.val = val

class MyClass(object):
    x = RevealAccess(10, '実数"x"')
    y = 5

m = MyClass()

# クラスが属性に対するデータディスクリプタを持つときはよばれる
print(m.x)
m.x = 20
print(m.x)
setattr(m, 'x', 30)
print(m.y)

In [ ]:
# Python内のオブジェクトは全て非ディスクリプタ
def function(): pass

print(hasattr(function, '__get__'))
print(hasattr(function, '__set__'))

print(hasattr(lambda: None, '__get__'))
print(hasattr(lambda: None, '__set__'))

#### ディスクリプタ - 属性の遅延評価

In [ ]:
# インスタンスから属性にアクセスされるまで初期化を遅延する
class InitOnAccess:
    def __init__(self, klass, *args, **kwargs) -> None:
        self.klass = klass
        self.args = args
        self.kwargs = kwargs
        self._initialized = None
    
    def __get__(self, instance, owner):
        if self._initialized is None:
            print('初期化')
            self._initialized = self.klass(*self.args, **self.kwargs)
        else:
            print('キャッシュ済み')
        return self._initialized


class MyClass:
    lazily_initialized = InitOnAccess(list, "argument")


m = MyClass()
print(m.lazily_initialized)
print(m.lazily_initialized)

### プロパティ

In [ ]:
class Rectangle:
    def __init__(self, x1, y1, x2, y2):
        self.x1, self.y1 = x1, y1
        self.x2, self.y2 = x2, y2
    
    def _width_get(self):
        return self.x2 - self.x1
    
    def _width_set(self, value):
        self.x2 = self.x1 + value
    
    def _height_get(self):
        return self.y2 - self.y1
    
    def _height_set(self, value):
        self.y2 = self.y1 + value
    
    # プロパティにfget, fset, (fdel). docを設定
    width = property(
        _width_get, _width_set,
        doc="長方形の幅"
    )

    height = property(
        _height_get, _height_set,
        doc="長方形の高さ"
    )

    def __repr__(self) -> str:
        return "{}({}, {}, {}, {})".format(
            self.__class__.__name__,
            self.x1, self.y1, self.x2, self.y2
        )

rectangle = Rectangle(10, 10, 25, 34)
print(rectangle.width, rectangle.height)

rectangle.width = 100
print(rectangle)

rectangle.height = 100
print(rectangle)

help(Rectangle)


# 継承したクラスにはプロパティ全体を派生クラス側でオーバーライド
class MetricRectangle(Rectangle):
    def _width_get(self):
        return "{} meters".format(self.x2 - self.x1)
    width = property(_width_get, Rectangle.width.fset)

print(MetricRectangle(0, 0, 100, 100).width)

In [ ]:
# プロパティをデコレータとして使用する
class Rectangle:
    def __init__(self, x1, y1, x2, y2):
        self.x1, self.y1 = x1, y1
        self.x2, self.y2 = x2, y2

    @property
    def width(self):
        """長方形の幅
        """
        return self.x2 - self.x1

    @width.setter
    def width(self, value):
        self.x2 = self.x1 + value

    @property
    def height(self):
        """長方形の高さ
        """
        return self.y2 - self.y1

    @height.setter
    def height(self, value):
        self.y2 = self.y1 + value

    def __repr__(self) -> str:
        return "{}({}, {}, {}, {})".format(
            self.__class__.__name__,
            self.x1, self.y1, self.x2, self.y2
        )

rectangle = Rectangle(10, 10, 25, 34)
print(rectangle.width, rectangle.height)

rectangle.width = 100
print(rectangle)

rectangle.height = 100
print(rectangle)

### スロット

In [ ]:
# __dict__を作成せず、プロパティを凍結することができる
class Frozen:
    __slots__ = ['ice', 'cream']

print('__dict__' in dir(Frozen))
print('ice' in dir(Frozen))

frozen = Frozen()
frozen.ice = True
frozen.cream = None
try:
    frozen.icy = True
except AttributeError as ex:
    print(ex)
